## Code for performing evaluation of $\Delta-$learning for the Hydra Challenge

First, Import usual packages. \
*Note*: To use the new FCHL19 represenation you need an extra step of configuration of the QML package. Please look at the documentation for details

In [1]:
import qml
import numpy as np
import pandas as pd
import os


In [2]:
def get_frequencies(file):
    with open(file,"r") as f:
        contents = f.read().splitlines()
        freq, smiles = contents[1].split()
    return float(freq)


In [3]:
def normalize_freq(freq):
    mean = np.mean(np.array(freq))
    std = np.std(np.array(freq))
    norm_freq = []
    for i in freq:
        new_freq = (i-mean)/std
        norm_freq.append(new_freq)
    return norm_freq, mean,std

Here, It reads the data to train the initial kernel

In [4]:
data_train = '/home/vazquez/Documents/Hydra/e_structures_final/train_no_test'
file_name_train = []
frequencies_train = []
freq_dict = {}
for path, dirs, files in os.walk(data_train):
    for file in files:
        if file.endswith(".xyz"):            
            file_path = path + "/" + file
            freq_dict[file] = get_frequencies(file_path)
            frequencies_train.append(get_frequencies(file_path))
            file_name_train.append(file)
        
dct = {'file':file_name_train, 'frequencies':frequencies_train}
df = pd.DataFrame(dct)

In [5]:
df

,file,frequencies
0,gens_288_gen9.xyz.out.xyz,3631.0391
1,gens_502_gen19.xyz.out.xyz,3602.7447
2,gens_75_gen19.xyz.out.xyz,3776.5611
3,gens_98-85_gen8.xyz.out.xyz,3764.4916
4,gens_434_gen3.xyz.out.xyz,3605.6133
...,...,...
208,gens_62_gen6.xyz.out.xyz,3554.0158
209,gens_434_gen2.xyz.out.xyz,3646.0809
210,gens_125132_gen6.xyz.out.xyz,3602.1282
211,gens_75_gen16.xyz.out.xyz,3725.5927


This dictionary contains the experimental frequencies

In [6]:
dct_f_exp = {'67-64-1.out.xyz':3538,'98-86-2.out.xyz':3536,'62-53-3.out.xyz':3524,'1191-95-3.out.xyz':3548,'132-64-9.out.xyz':3623,'2406-25-9.out.xyz':3484,
           '288-32-4.out.xyz':3458,'611-20-1.out.xyz':3595,'98-85-1.out.xyz':3620,'327-54-8.out.xyz':3647}    

Frequencies are normalized and the value of the mean and the standard deviation is printed


In [7]:
new_freq = normalize_freq(df['frequencies'])
for i in range(len(df['file'])):
    freq_dict[df['file'][i]]=new_freq[0][i]

print(new_freq[1])
print(new_freq[2])

3633.5051751173705
76.41647271169035


Load modules for FCHL19

In [8]:
from qml.math import cho_solve

from qml.representations import generate_fchl_acsf

from qml.kernels import get_local_kernel
from qml.kernels import get_local_symmetric_kernel

Here the representation for the molecules in training set is generated. It also appends as property the normalized frequency. 

In [9]:
mols = []
Qall = []
for xyz_file in df['file']:
    mol = qml.Compound(xyz=data_train+'/'+xyz_file)
    mol.properties = freq_dict[xyz_file]
    mol.representation = generate_fchl_acsf(mol.nuclear_charges, mol.coordinates, elements = [1,6,7,8,9,16],
        nRs2=22, nRs3=17, nFourier=1, eta2=0.41, eta3=0.97, zeta=np.pi, rcut=8.0, acut=8.0,
        two_body_decay=2.4, three_body_decay=2.4, three_body_weight=45.8, gradients=False,pad=33)
    Qall.append(mol.nuclear_charges)
    mols.append(mol)

Arrays for the kernel. It is one for the representation, one for nuclear charges and one for the properties.


In [10]:
X  = np.array([mol.representation for mol in mols])
Q  = np.array([mol.nuclear_charges for mol in mols])
Y = np.array([mol.properties for mol in mols])

/opt/intel/oneapi/intelpython/latest/lib/python3.7/site-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


Basic hyperparameters of the kernel. This might be optimized. Thne the local symmetric kernel is obtained and the matrix of coefficients is obtained by Cholesky decomposition

In [11]:
sigma = 1.2
llambda = 1e-10

K = get_local_symmetric_kernel(X, Q, sigma)

# Solve alpha
alpha = cho_solve(K,Y, l2reg=llambda)

Here starts the validation of the generated kernel. The molecules to test are the ones with available experimental data. It consider the experimental frequencies by read them from the dictionary define adobe.


In [22]:
data_test = '/home/vazquez/Documents/Hydra/e_structures_final/val_exp'
file_name_test = []
frequencies_test = []
test_freq_dict = {}
for path, dirs, files in os.walk(data_test):
    for file in files:
        if file.endswith(".xyz"):            
            file_path = path + "/" + file
            frequencies_test.append(dct_f_exp[file])
            test_freq_dict[file] = dct_f_exp[file]
            file_name_test.append(file)

dct1 = {'file':file_name_test,'exp_freq':frequencies_test}
df1 = pd.DataFrame(dct1)

Experimental frequencies are normalized

In [23]:
new_freq_test = normalize_freq(df1['exp_freq'])
for i in range(len(df1['file'])):
    test_freq_dict[df1['file'][i]]=new_freq_test[0][i]

print(test_freq_dict)

{'288-32-4.out.xyz': -1.6791937587614252, '67-64-1.out.xyz': -0.32636897828898037, '98-86-2.out.xyz': -0.3601895978007915, '62-53-3.out.xyz': -0.5631133148716582, '98-85-1.out.xyz': 1.0602764216952756, '2406-25-9.out.xyz': -1.2395257051078805, '327-54-8.out.xyz': 1.5168547851047256, '1191-95-3.out.xyz': -0.15726588072992478, '132-64-9.out.xyz': 1.111007350962992, '611-20-1.out.xyz': 0.6375186777976365}


The descriptor for the test molecules is generated.

In [24]:
mols_test = []
Qall_test = []
for xyz_file in df1['file']:
    mol = qml.Compound(xyz=data_test+'/'+xyz_file)
    mol.properties = test_freq_dict[xyz_file]
    mol.representation = generate_fchl_acsf(mol.nuclear_charges, mol.coordinates, elements = [1,6,7,8,9,16],
        nRs2=22, nRs3=17, nFourier=1, eta2=0.41, eta3=0.97, zeta=np.pi, rcut=8.0, acut=8.0,
        two_body_decay=2.4, three_body_decay=2.4, three_body_weight=45.8, gradients=False,pad=33)
    Qall_test.append(mol.nuclear_charges)
    mols_test.append(mol)

An array of the test molecules, charges and the properties is also generated

In [25]:
Xs = np.array([mol.representation for mol in mols_test])
Qs = np.array([mol.nuclear_charges for mol in mols_test])
Ys = np.array([mol.properties for mol in mols_test])

/opt/intel/oneapi/intelpython/latest/lib/python3.7/site-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [26]:
# Calculate test prediction kernel
Ks = get_local_kernel(X, Xs, Q, Qs, sigma)
Yss = np.dot(Ks, alpha)

In [27]:
rescaled_Yss = []
for i in Yss:
    print('Frequency', (new_freq_test[2]*i)+new_freq_test[1])
    rescaled_Yss.append(new_freq_test[2]*i+new_freq_test[1])

Frequency 3512.7005003433005
Frequency 3548.197100150081
Frequency 3538.03415099271
Frequency 3530.417817188364
Frequency 3662.025098634388
Frequency 3439.624604236679
Frequency 3672.7702826029526
Frequency 3555.0695296033123
Frequency 3652.4738747436904
Frequency 3607.7969917142786


MAE with respect to the experimental values using the normalization of the experimental values.

In [29]:
print(np.mean(np.abs(df1['exp_freq']-rescaled_Yss)))
Error = df1['exp_freq']-rescaled_Yss

23.48607417363987


In [30]:
dc_plot = {'files':df1['file'],'Frequencies_exp':df1['exp_freq'],'Frequencies_predict':rescaled_Yss,'Error':Error}
df_plot = pd.DataFrame(dc_plot)

In [31]:
df_plot

,files,Frequencies_exp,Frequencies_predict,Error
0,288-32-4.out.xyz,3458,3512.700500,-54.700500
1,67-64-1.out.xyz,3538,3548.197100,-10.197100
2,98-86-2.out.xyz,3536,3538.034151,-2.034151
3,62-53-3.out.xyz,3524,3530.417817,-6.417817
4,98-85-1.out.xyz,3620,3662.025099,-42.025099
5,2406-25-9.out.xyz,3484,3439.624604,44.375396
6,327-54-8.out.xyz,3647,3672.770283,-25.770283
7,1191-95-3.out.xyz,3548,3555.069530,-7.069530
8,132-64-9.out.xyz,3623,3652.473875,-29.473875
9,611-20-1.out.xyz,3595,3607.796992,-12.796992


# $\Delta$-Learning

Here I start with the $\Delta$-learning approach

In [32]:
#Delta Learning
dct_error = {}
for i in range(10):
    dct_error[df1['file'][i]] = df_plot['Error'][i]

Generate the representation for the molecules in the validation set. 

In [33]:
Qall_delta = []    
mols_delta = []
for xyz_file in df1['file']:
    mol = qml.Compound(xyz=data_test+'/'+xyz_file)
    mol.properties = dct_error[xyz_file]
    mol.representation = generate_fchl_acsf(mol.nuclear_charges, mol.coordinates, elements = [1,6,7,8,9,16],
        nRs2=22, nRs3=17, nFourier=1, eta2=0.41, eta3=0.97, zeta=np.pi, rcut=8.0, acut=8.0,
        two_body_decay=2.4, three_body_decay=2.4, three_body_weight=45.8, gradients=False,pad=33)
    Qall_test.append(mol.nuclear_charges)
    mols_delta.append(mol)
    
Xd = np.array([mol.representation for mol in mols_delta])
Qd = np.array([mol.nuclear_charges for mol in mols_delta])
Yd = np.array([mol.properties for mol in mols_delta])

/opt/intel/oneapi/intelpython/latest/lib/python3.7/site-packages/ipykernel_launcher.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  del sys.path[0]


Solve the kernel equation for the $\Delta$

In [34]:
#Solving delta equations
sigma = 1.2
llambda = 1e-10

K_delta = get_local_symmetric_kernel(Xd, Qd, sigma)

# Solve alpha
alpha_d = cho_solve(K_delta,Yd, l2reg=llambda)

Testing with the molecules for the challenge, the kernel approach. First read the files and create the respective arrays.

In [35]:
# Here starts the part of the testing molecules

data_predict = '/home/vazquez/Documents/Hydra/e_structures_final/test'
file_name_predict = []
frequencies_predict = []
freq_dict_predict = {}
for path, dirs, files in os.walk(data_predict):
    for file in files:
        if file.endswith(".xyz"):            
            file_path = path + "/" + file
            freq_dict_predict[file] = get_frequencies(file_path)
            frequencies_predict.append(get_frequencies(file_path))
            file_name_predict.append(file)

dct2 = {'file':file_name_predict,'frequencies_calc':frequencies_predict}
df2 = pd.DataFrame(dct2)

Generate the FCHL representation for the molecules of the test set

In [36]:
mols_predict = []
Qall_predict = []
for xyz_file in df2['file']:
    mol = qml.Compound(xyz=data_predict+'/'+xyz_file)
    mol.properties = freq_dict_predict[xyz_file]
    mol.representation = generate_fchl_acsf(mol.nuclear_charges, mol.coordinates, elements = [1,6,7,8,9,16],
        nRs2=22, nRs3=17, nFourier=1, eta2=0.41, eta3=0.97, zeta=np.pi, rcut=8.0, acut=8.0,
        two_body_decay=2.4, three_body_decay=2.4, three_body_weight=45.8, gradients=False,pad=33)
    Qall_predict.append(mol.nuclear_charges)
    mols_predict.append(mol)

In [37]:
Xp = np.array([mol.representation for mol in mols_predict])
Qp = np.array([mol.nuclear_charges for mol in mols_predict])
Yp = np.array([mol.properties for mol in mols_predict])

/opt/intel/oneapi/intelpython/latest/lib/python3.7/site-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


Do the evaluation of the molecules

In [38]:
# Calculate test prediction kernel
Kp = get_local_kernel(X, Xp, Q, Qp, sigma)
Ypred = np.dot(Kp, alpha)

In [39]:
rescaled_Ypred = []
for i in Ypred:
    print('Frequency', (new_freq_test[2]*i)+new_freq_test[1])
    rescaled_Ypred.append(new_freq_test[2]*i+new_freq_test[1])

Frequency 3620.6513021574233
Frequency 3602.972481428225
Frequency 3616.700635023064
Frequency 3602.307194865952
Frequency 3510.3301629626712
Frequency 3578.8541100829466
Frequency 3440.6887484570893
Frequency 3514.483928995163
Frequency 3548.8399599449663
Frequency 3543.9458554583553


In [40]:
#Calculate the delta difference
Kpd = get_local_kernel(Xd, Xp, Qd, Qp, sigma)
Ypred_delta = np.dot(Kpd, alpha_d)

In [41]:
rescale_and_delta = []
for i in range(len(rescaled_Ypred)):
    print('New freq:', rescaled_Ypred[i] + Ypred_delta[i])
    rescale_and_delta.append(rescaled_Ypred[i]+ Ypred_delta[i])

New freq: 3585.438278202633
New freq: 3589.4484237621514
New freq: 3585.4644562400827
New freq: 3570.308625874118
New freq: 3479.684426432184
New freq: 3523.9854466137253
New freq: 3405.4904812257914
New freq: 3541.6057506171373
New freq: 3508.954866810482
New freq: 3511.56269947435


In [42]:
results_l = [3597,3611,3507,3591,3524,3649,3454,3503,3492,3491]

In [43]:
pred_dict = {'File':df2['file'].to_numpy(),'Experimental':results_l,'Freq_harmonic':df2['frequencies_calc'].to_numpy(), 'Predicted_by_kernel':rescaled_Ypred, 
            'Predicted+delta':rescale_and_delta}
df_pred = pd.DataFrame(pred_dict)

In [44]:
df_pred

,File,Experimental,Freq_harmonic,Predicted_by_kernel,Predicted+delta
0,125132-75-4.out.xyz,3597,3739.5545,3620.651302,3585.438278
1,434-45-7.out.xyz,3611,3710.7747,3602.972481,3589.448424
2,110-01-0.out.xyz,3507,3595.0460,3616.700635,3585.464456
3,50-0-0.out.xyz,3591,3692.3172,3602.307195,3570.308626
4,547-64-8.out.xyz,3524,3610.2268,3510.330163,3479.684426
5,75-89-8.out.xyz,3649,3607.1661,3578.854110,3523.985447
6,110-86-1.out.xyz,3454,3498.3457,3440.688748,3405.490481
7,502-49-8.out.xyz,3503,3578.1188,3514.483929,3541.605751
8,80-73-9.out.xyz,3492,3574.8747,3548.839960,3508.954867
9,109-99-9.out.xyz,3491,3575.1894,3543.945855,3511.562699


In [48]:
old_results = pd.read_csv('final_kernel.csv')
Error_old = old_results['Experiment']-old_results['Predicted+delta']
Error_new = old_results['Experiment']-df_pred['Predicted+delta']

In [50]:
new_results_dct = {'Molecule':df_pred['File'],'Exp Freq.':old_results['Experiment'],'Initial Subm':old_results['Predicted+delta'],'Error Initial':Error_old,
                   'Model1':df_pred['Predicted_by_kernel'],'Corrected':df_pred['Predicted+delta'],'Error Correc':Error_new}
new_results_df = pd.DataFrame(new_results_dct)

In [51]:
new_results_df.to_csv('corrected_results_with_kernel.csv',index=False,float_format='%.1f')